# Домашнее задание № 11. Генерация текста
## Задание 1 (10 баллов).
В семинаре мы работали с датасетами инструкций alpaca и dolly. Они англоязычные. В домашке вам нужно создать аналогичный датасет на русском языке и обучить аналогичную модель на этом датасете. В качестве итогового результата у вас должна получится модель, которая может связно отвечать на русскоязычные инструкции на русском языке. Приведите как минимум три разных примера. Правильность ответов не так важна, так как вы скорее всего будете использовать небольшие модели, но текст должен быть не рандомным.

Русскоязычный датасет инструкций должен быть больше 5 тысяч примеров. Он может быть основнован на alpaca/dolly (например, вы можете просто прогнать все через переводную модель, которая была на семинаре, или даже google translate). Или вы можете придумать способ создать аналогичный датасет каким-то другим способом (переделать открытые датасеты с помощью правил). Датасет может быть не уникальным, можно скооперироваться с одногруппниками и сделать один датасет на всех.

Вы можете попробовать дообучать любую небольшую decoder-only модель. Скорее всего лучше всего будут работать модели, изначально обученные на русском языке (rugpt например). Но возможно даже модели вроде opt можно будет дообучить на русскоязычных инструкциях.

Это задание гораздно менее определенное, по сравнению с предыдущими. Поэтому не стесняйтесь задавать дополнительные вопросы в чате или лично, если у вас возникнут трудности.

In [ ]:
# %pip install pandas transformers tokenizers datasets xformers

Скачаем датасет

In [ ]:
!pip install zstandard jsonlines

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 22.3 MB/s eta 0:00:00


In [1]:
import copy
import logging
from dataclasses import dataclass, field
from typing import Optional, Dict, Sequence
import json, os
import torch
import transformers
from torch.utils.data import Dataset
from transformers import Trainer

device = torch.device("cuda")
os.environ['CURL_CA_BUNDLE'] = ''
HUGGING_FACE_API_KEY = os.environ.get("HUGGING_FACE_API_KEY")

# import utils

c:\Users\Polifolli\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# для моего первого аккаунта
with open('/content/drive/MyDrive/КомпЛинг/ДЗ 11/ru_turbo_alpaca.jsonl') as f:
    data = [json.loads(line) for line in f]

In [ ]:
# для моего второго аккаунта
with open('/content/drive/MyDrive/КомпЛинг/ru_turbo_alpaca.jsonl') as f:
    data = [json.loads(line) for line in f]

In [4]:
# для моего компа
with open('ru_turbo_alpaca.jsonl', encoding = 'utf-8') as f:
    data = [json.loads(line) for line in f]

In [5]:
attributes_to_keep = ["instruction", "input", "output"]

In [6]:
rus_alpaca = [{k: v for k, v in example.items() if k in attributes_to_keep} for example in data]
rus_alpaca[:5]

[{'instruction': 'Опишите, как сделать горшок из глины.',
  'input': '',
  'output': 'Для изготовления горшка из глины сначала необходимо взять глину и размешать ее водой до состояния пластилина. Затем нужно сформировать глиняный комок нужной формы и тщательно откачать влагу из глины, чтобы она стала твердой. Горшок нужно обжечь в духовке при высокой температуре, чтобы он стал прочным и непроницаемым.'},
 {'instruction': 'Опиши процесс изготовления торта. Укажи ингредиенты, необходимые для этого.',
  'input': '',
  'output': 'Для изготовления торта необходимы следующие ингредиенты: мука, сахар, яйца, молоко, сливочное масло, разрыхлитель, ванилин. Сначала нужно смешать муку с сахаром и добавить яйца, молоко, разрыхлитель и ваниль. Затем тесто надо хорошо перемешать и разделить на несколько частей. Каждую часть нужно запекать в духовке до готовности. После этого торт надо собрать, сложив слои пирога вместе с начинкой и украшениями.'},
 {'instruction': 'Напиши аналог данной пословицы на 

В нем каждый пример это инструкция, опциональный контекст и ответ.
Для модели эти примеры еще оборачиваются в специальный промпт, который говорит модели, что она должна следовать инструкциям.

In [10]:
IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = "[PAD]"
DEFAULT_EOS_TOKEN = "</s>"
DEFAULT_BOS_TOKEN = "</s>"
DEFAULT_UNK_TOKEN = "</s>"
PROMPT_DICT = {
    "prompt_input": (
        "Ниже приведена инструкция, описывающая задачу, инструкции соответствует input, который приводит дополнительный контекст. "
        "Напишите ответ, который соответствующим образом выполняет запрос.\n\n"
        "### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:"
    ),
    "prompt_no_input": (
        "Ниже приведена инструкция, описывающая задание. "
        "Напишите ответ, который соответствующим образом выполняет запрос.\n\n"
        "### Instruction:\n{instruction}\n\n### Response:"
    ),
}


Давайте попробуем дообучить модель от facebook - opt (она открытыя и устроена как LLama и GPT - это декодер онли модель)

Далее код взят из гитхаба Alpaca и он на торче, но если поизучать его, то будет видно, что тут происходят те же манипуляции, что мы делали раньше (превращение токенов в индексы и паддинг/урезание последовательностей)

In [11]:
def _tokenize_fn(strings: Sequence[str], tokenizer: transformers.PreTrainedTokenizer) -> Dict:
    """Tokenize a list of strings."""
    tokenized_list = [
        tokenizer(
            text,
            return_tensors="pt",
            max_length=tokenizer.model_max_length,
            truncation=True,
        )
        for text in strings
    ]
    input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
    input_ids_lens = labels_lens = [
        tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
    ]
    return dict(
        input_ids=input_ids,
        labels=labels,
        input_ids_lens=input_ids_lens,
        labels_lens=labels_lens,
    )

In [12]:
def preprocess(
    sources: Sequence[str],
    targets: Sequence[str],
    tokenizer: transformers.PreTrainedTokenizer,
) -> Dict:
    """Preprocess the data by tokenizing."""
    examples = [s + t for s, t in zip(sources, targets)]
    examples_tokenized, sources_tokenized = [_tokenize_fn(strings, tokenizer) for strings in (examples, sources)]
    input_ids = examples_tokenized["input_ids"]
    labels = copy.deepcopy(input_ids)
    for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
        label[:source_len] = IGNORE_INDEX
    return dict(input_ids=input_ids, labels=labels)

Далее это оборачивается к классы, которые предобрабатывают данные к формату huggingface.

In [13]:
class SupervisedDataset(Dataset):
    """Dataset for supervised fine-tuning."""

    def __init__(self, data_path: str, tokenizer: transformers.PreTrainedTokenizer):
        super(SupervisedDataset, self).__init__()
        logging.warning("Loading data...")
        #list_data_dict = json.load(open(data_path))
        list_data_dict = rus_alpaca

        logging.warning("Formatting inputs...")
        prompt_input, prompt_no_input = PROMPT_DICT["prompt_input"], PROMPT_DICT["prompt_no_input"]
        sources = [
            prompt_input.format_map(example) if example.get("input", "") != "" else prompt_no_input.format_map(example)
            for example in list_data_dict
        ]
        targets = [f"{example['output']}{tokenizer.eos_token}" for example in list_data_dict]

        logging.warning("Tokenizing inputs... This may take some time...")
        data_dict = preprocess(sources, targets, tokenizer)

        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return dict(input_ids=self.input_ids[i], labels=self.labels[i])


@dataclass
class DataCollatorForSupervisedDataset(object):
    """Collate examples for supervised fine-tuning."""

    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=IGNORE_INDEX)
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )

Загружаем модель

## Другая модель

In [ ]:
# model_name = 'facebook/opt-350m'
model_name_2 = "inkoziev/rugpt_chitchat"
model_2 = transformers.AutoModelForCausalLM.from_pretrained(
        model_name_2,
        max_length=512,
        cache_dir="huggingface_cache",
    )

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_name_2,
    cache_dir="huggingface_cache",
    model_max_length=512,
    padding_side="right",
    use_fast=False,
)

In [16]:
train_args = transformers.TrainingArguments(learning_rate=1e-5,
                 num_train_epochs=1,
                 per_device_train_batch_size=2,
                 gradient_accumulation_steps=1,
                 evaluation_strategy='no',
                 weight_decay=0.,
                 warmup_ratio=0.03,
                 lr_scheduler_type="cosine",
                 save_strategy='no',
                 logging_steps=1000,
                 output_dir="opt125_instruct_ft")

In [17]:
train_dataset = SupervisedDataset(tokenizer=tokenizer, data_path="ru_turbo_alpaca.jsonl")
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)

In [18]:
trainer = Trainer(model=model_2,
                 tokenizer=tokenizer,
                 args=train_args,
                 train_dataset=train_dataset,
                 eval_dataset=None,
                 data_collator=data_collator)

/home/users/a.perehodov/miniconda3/envs/env_workspace/lib/python3.11/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [19]:
trainer.train()

  0%|          | 0/14911 [00:00<?, ?it/s]

  7%|▋         | 1001/14911 [03:06<42:54,  5.40it/s]

{'loss': 3.2727, 'grad_norm': 4.297895908355713, 'learning_rate': 9.964101131324918e-06, 'epoch': 0.07}


 13%|█▎        | 2001/14911 [06:11<43:11,  4.98it/s]

{'loss': 2.6903, 'grad_norm': 3.964296579360962, 'learning_rate': 9.718557674181557e-06, 'epoch': 0.13}


...

 94%|█████████▍| 14001/14911 [43:03<02:35,  5.87it/s]

{'loss': 2.2463, 'grad_norm': 5.431273460388184, 'learning_rate': 9.757579538917539e-08, 'epoch': 0.94}


100%|██████████| 14911/14911 [45:52<00:00,  5.42it/s]

{'train_runtime': 2752.0575, 'train_samples_per_second': 10.836, 'train_steps_per_second': 5.418, 'train_loss': 2.408617751458131, 'epoch': 1.0}


TrainOutput(global_step=14911, training_loss=2.408617751458131, metrics={'train_runtime': 2752.0575, 'train_samples_per_second': 10.836, 'train_steps_per_second': 5.418, 'train_loss': 2.408617751458131, 'epoch': 1.0})

Сохраним модель

In [20]:
trainer.save_model('rugpt_1')

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512}


И давайте попробуем ее на том же тексте

In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [8]:
MODEL_NAME = 'rugpt_1'

In [9]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, model_max_length=512, max_length=512)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, max_length=512)

In [11]:
def predict_for_instruction(instruction, text, model):
    text = text.replace('\n', ' ')
    prompt = ("Ниже приведена инструкция, описывающая задачу, инструкции соответствует input, который приводит дополнительный контекст. "
              "Напишите ответ, который соответствующим образом выполняет запрос.\n\n"
              f"### Instruction:\n{instruction}\n\n### Input:\n{text}\n\n### Response:")

    inputs = tokenizer([prompt],
                        return_tensors="pt", padding=True)

    output_sequences = model.generate(
        # this parameters are also important but you can read about them in the docs and just try changing them
        num_beams=1,
#         temperature=0.4,
#         max_length=100,
        max_new_tokens=20,
#         no_repeat_ngram_size=3,
    #     repetition_penalty= 5.0,
    #     length_penalty=0.01,
    #     early_stopping=True,
    #     do_sample=True,
    #     top_k=30,
    #     top_p=0.8,
        early_stopping=True,
    #     num_return_sequences=3,
        num_return_sequences= 1,
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        do_sample=False,  # disable sampling to test if batching affects output
    )
    summaries = tokenizer.batch_decode(output_sequences[:,len(inputs[0]):], skip_special_tokens=True)
    return summaries[0]

In [12]:
text = """
Барсуки, зарывшиеся под железнодорожными путями, остановили движение поездов на севере и юге Нидерландов, что привело к длительным отменам по крайней мере на двух линиях.
Во вторник днем все поезда были остановлены на оживленной линии между южными городами Ден Босх и Бокстел после того, как животные зарылись в дамбу, несущую рельсы. Национальная железнодорожная компания сообщила, что линия не будет работать как минимум неделю.
Подкоп означает, что "рельсы могут проседать, и тогда безопасность движения поездов больше не гарантируется", - говорится в заявлении ProRail, компании, обслуживающей железнодорожную сеть Нидерландов.
Ранее в этом месяце барсуки также зарылись под рельсы в районе северной деревни Молкверум в провинции Фрисландия, что вывело линию из эксплуатации до следующего месяца, пока рабочие будут добиваться разрешения на перемещение животных.
Барсуки являются охраняемыми животными в Нидерландах, поэтому железнодорожники должны получить разрешение на их перемещение или нарушение их среды обитания, прежде чем начать ремонт.
"""

In [26]:
instruction = "Дайте краткое изложение этого текста."
predict_for_instruction(instruction, text, model)

/home/users/a.perehodov/miniconda3/envs/env_workspace/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:535: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Барсуки, зарывшиеся под железнодорожными поездами, остановили движение поездов на севере и юге голланд'

In [27]:
instruction = "Give a very short summary of this text."
predict_for_instruction(instruction, text, model)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Барсуки, зарывшиеся под железнодорожными поездами, остановили движение поездов на севере и юге голланд'

In [28]:
instruction = "Напишите заголовок для следующего текста."
predict_for_instruction(instruction, text, model)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Барсуки, зарывшиеся под железнодорожными поездами, остановили движение на севере и юге голландской'

In [29]:
instruction = "О каком животном этот текст?"
predict_for_instruction(instruction, text, model)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Барсуки - это животные, которые зарылись под железнодорожными поездами на севере и юге Франции'

In [18]:
text2 = """
В Индонезии началось извержение вулкана Руанг.
Столб пепла поднялся на высоту трех километров. Также в момент извержения было зафиксировано землетрясение магнитудой 3,2.
Из-за извержения более 800 жителей близлежащих деревень были вынуждены эвакуироваться. Сейчас запрещено находиться в радиусе четырех километров от вулкана
"""

In [19]:
instruction = "Дайте краткое изложение этого текста."
predict_for_instruction(instruction, text2, model)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'В Индонезию произошло извержение вулкана Руанг. Столб пепла поднялся на высоту 3 километров.'

In [26]:
text3 = ''

In [28]:
instruction = "Придумай пять разных кличек для собак"
predict_for_instruction(instruction, text3, model)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Собачка, Кошка, Собака, Собака, Собака.\n'

In [29]:
instruction = "Расскажи рецепт лазаньи"
predict_for_instruction(instruction, text3, model)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Для приготовления лазаньи вам понадобятся: тесто, яйца, лук, чеснок, оливковое масло,'